## week 10 assignment
### 201721402 나동현

'제74주년 경찰의 날 기념식 축사' 전문을 읽고 TF-IDF를 계산하세요.

전문은 http://www.korea.kr/archive/speechView.do?newsId=132031636 에서 읽을 수 있고,

해당 사이트에서 텍스트만 파일로 저장한 후 사용한다.



* 1) DataFrame을 생성

* 2) 단어로 분리해서, 출력

* 3) 불용어 구성, 출력 - 축사 전문에서 한글자로 된 단어를 찾아내 스스로 구성

* 4) 불용어 제거하고, 출력

* 5) TF-IDF를 계산하고, 출력

* 6) TF-IDF 컬럼을 features로 구성, 출력

In [113]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [114]:
import os
_file=os.path.join('data','week10_assignment_textFile.txt')
_rdd=spark.sparkContext.textFile(_file)
_col12 = _rdd.map(lambda l: l.split('\n'))

1) DataFrame을 생성

In [115]:
myDf=spark.createDataFrame(_col12, ['sent'])

2) 단어로 분리해서, 출력



In [116]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="sent", outputCol="words")
tokDf = tokenizer.transform(myDf)
for r in tokDf.select("words").take(41):
    print (r)

Row(words=['존경하는', '국민', '여러분,', '경찰관', '여러분,', '일흔네', '돌', '‘경찰의', '날’입니다.'])
Row(words=['국민의', '안전을', '위해', '밤낮없이', '애쓰시는', '전국의', '15만', '경찰관', '여러분께', '먼저', '감사를', '드립니다.', '전몰·순직', '경찰관들의', '고귀한', '희생에', '경의를', '표합니다.', '유가족', '여러분께', '위로의', '마음을', '전합니다.'])
Row(words=['오늘', '홍조근정훈장을', '받으신', '중앙경찰학교장', '이은정', '치안감님,', '근정포장을', '받으신', '광주남부경찰서', '김동현', '경감님을', '비롯한', '수상자', '여러분께', '각별한', '축하와', '감사를', '드립니다.', '또한', '경찰', '영웅으로', '추서되신', '차일혁,', '최중락님께', '국민의', '사랑을', '전해드립니다.'])
Row(words=['사랑하는', '경찰관', '여러분,'])
Row(words=['여러분의', '헌신적', '노력으로', '우리의', '치안은', '참', '좋아졌습니다.', '지난해', '범죄', '발생은', '2015년에', '비해', '15.1%', '줄었습니다.', '같은', '기간', '교통사고', '사망자는', '18.2%', '감소했습니다.'])
Row(words=['치안의', '개선은', '국민의', '체감으로', '나타나고', '있습니다.', '올해', '상반기', '국민의', '체감안전도는', '74.5점으로', '역대', '최고를', '기록했습니다.', '범죄안전도는', '처음으로', '80점을', '넘었습니다.'])
Row(words=['한국을', '찾는', '외국', '관광객들도', '우리의', '치안을', '가장', '좋게', '평가합니다.', '한국의', '무엇이', '좋았느냐는', '물음에', '외국', '관광객들은', '7년', '연속으로', '치안이', '가장', 

3) 불용어 구성, 출력 - 축사 전문에서 한글자로 된 단어를 찾아내 스스로 구성

In [117]:
__stopwords=list()
_mystopwords=[u"참", u"더", u"그", u"등", u"법", u"큰", u"돌"]
for e in _mystopwords:
    __stopwords.append(e)
stop.setStopWords(__stopwords)

StopWordsRemover_f381d1a38713

In [118]:
for e in stop.getStopWords():
    print (e, end="/")

참/더/그/등/법/큰/돌/

4) 불용어 제거하고, 출력

In [119]:
from pyspark.ml.feature import RegexTokenizer

re = RegexTokenizer(inputCol="sent", outputCol="wordsReg", pattern="\\s+")
reDf=re.transform(myDf)

In [120]:
stopDf=stop.transform(reDf)
for r in stopDf.select("nostops").take(41):
    print (r)

Row(nostops=['존경하는', '국민', '여러분,', '경찰관', '여러분,', '일흔네', '‘경찰의', '날’입니다.'])
Row(nostops=['국민의', '안전을', '위해', '밤낮없이', '애쓰시는', '전국의', '15만', '경찰관', '여러분께', '먼저', '감사를', '드립니다.', '전몰·순직', '경찰관들의', '고귀한', '희생에', '경의를', '표합니다.', '유가족', '여러분께', '위로의', '마음을', '전합니다.'])
Row(nostops=['오늘', '홍조근정훈장을', '받으신', '중앙경찰학교장', '이은정', '치안감님,', '근정포장을', '받으신', '광주남부경찰서', '김동현', '경감님을', '비롯한', '수상자', '여러분께', '각별한', '축하와', '감사를', '드립니다.', '또한', '경찰', '영웅으로', '추서되신', '차일혁,', '최중락님께', '국민의', '사랑을', '전해드립니다.'])
Row(nostops=['사랑하는', '경찰관', '여러분,'])
Row(nostops=['여러분의', '헌신적', '노력으로', '우리의', '치안은', '좋아졌습니다.', '지난해', '범죄', '발생은', '2015년에', '비해', '15.1%', '줄었습니다.', '같은', '기간', '교통사고', '사망자는', '18.2%', '감소했습니다.'])
Row(nostops=['치안의', '개선은', '국민의', '체감으로', '나타나고', '있습니다.', '올해', '상반기', '국민의', '체감안전도는', '74.5점으로', '역대', '최고를', '기록했습니다.', '범죄안전도는', '처음으로', '80점을', '넘었습니다.'])
Row(nostops=['한국을', '찾는', '외국', '관광객들도', '우리의', '치안을', '가장', '좋게', '평가합니다.', '한국의', '무엇이', '좋았느냐는', '물음에', '외국', '관광객들은', '7년', '연속으로', '치안이', '가

5) TF-IDF를 계산하고, 출력

In [121]:
newList = _col12.take(41)

In [122]:
from functools import reduce
doc = reduce(lambda x,y: x+y, newList)

In [123]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=1.0, stop_words='english',norm = None)

In [124]:
print (vectorizer.fit_transform(doc))

  (0, 257)	3.3978952727983707
  (0, 53)	2.2992829841302607
  (0, 251)	3.3978952727983707
  (0, 45)	2.2992829841302607
  (0, 212)	4.963209081848431
  (0, 80)	3.3978952727983707
  (0, 278)	3.3978952727983707
  (1, 270)	3.3978952727983707
  (1, 139)	3.3978952727983707
  (1, 228)	3.3978952727983707
  (1, 233)	3.3978952727983707
  (1, 325)	3.3978952727983707
  (1, 42)	3.3978952727983707
  (1, 346)	3.3978952727983707
  (1, 59)	3.3978952727983707
  (1, 46)	3.3978952727983707
  (1, 194)	3.3978952727983707
  (1, 267)	3.3978952727983707
  (1, 134)	2.7047480922384253
  (1, 15)	2.992430164690206
  (1, 144)	2.992430164690206
  (1, 213)	5.984860329380412
  (1, 1)	3.3978952727983707
  (1, 266)	3.3978952727983707
  (1, 206)	3.3978952727983707
  :	:
  (19, 197)	3.3978952727983707
  (19, 92)	3.3978952727983707
  (19, 55)	3.3978952727983707
  (19, 41)	3.3978952727983707
  (19, 19)	3.3978952727983707
  (19, 50)	3.3978952727983707
  (19, 158)	3.3978952727983707
  (19, 240)	3.3978952727983707
  (19, 247)	3.

In [125]:
vectorizer.vocabulary_

{'존경하는': 278,
 '국민': 80,
 '여러분': 212,
 '경찰관': 45,
 '일흔네': 251,
 '경찰의': 53,
 '입니다': 257,
 '국민의': 84,
 '안전을': 201,
 '위해': 232,
 '밤낮없이': 165,
 '애쓰시는': 206,
 '전국의': 266,
 '15만': 1,
 '여러분께': 213,
 '먼저': 144,
 '감사를': 15,
 '드립니다': 134,
 '전몰': 267,
 '순직': 194,
 '경찰관들의': 46,
 '고귀한': 59,
 '희생에': 346,
 '경의를': 42,
 '표합니다': 325,
 '유가족': 233,
 '위로의': 228,
 '마음을': 139,
 '전합니다': 270,
 '오늘': 220,
 '홍조근정훈장을': 340,
 '받으신': 159,
 '중앙경찰학교장': 290,
 '이은정': 243,
 '치안감님': 315,
 '근정포장을': 96,
 '광주남부경찰서': 75,
 '김동현': 103,
 '경감님을': 38,
 '비롯한': 175,
 '수상자': 192,
 '각별한': 14,
 '축하와': 309,
 '또한': 137,
 '경찰': 43,
 '영웅으로': 219,
 '추서되신': 308,
 '차일혁': 300,
 '최중락님께': 307,
 '사랑을': 178,
 '전해드립니다': 271,
 '사랑하는': 179,
 '여러분의': 215,
 '헌신적': 338,
 '노력으로': 112,
 '우리의': 227,
 '치안은': 317,
 '좋아졌습니다': 280,
 '지난해': 292,
 '범죄': 167,
 '발생은': 161,
 '2015년에': 5,
 '비해': 176,
 '15': 0,
 '줄었습니다': 288,
 '같은': 21,
 '기간': 97,
 '교통사고': 76,
 '사망자는': 180,
 '18': 2,
 '감소했습니다': 18,
 '치안의': 319,
 '개선은': 23,
 '체감으로': 304,
 '나타나고': 108,
 '있습니다': 259,
 

6) TF-IDF 컬럼을 features로 구성, 출력

In [126]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash")
hashDf = hashTF.transform(stopDf)
hashDf.select("nostops", "hash").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|nostops                                                                                                   